In [5]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

In [6]:
base = pd.read_csv("stage3.csv")

In [7]:
pd.set_option('display.max_columns', None)
base.head()

,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district
0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,Shot - Wounded/Injured||Mass Shooting (4+ vict...,40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||3::Male||4::Female,0::Julian Sims,NaN,0::Arrested||1::Injured||2::Injured||3::Injure...,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://pittsburgh.cbslocal.com/2013/01/01/4-pe...,NaN,NaN
1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,http://www.dailybulletin.com/article/zz/201301...,False,43.0,NaN,NaN,"Shot - Wounded/Injured||Shot - Dead (murder, a...",33.9090,NaN,-118.3330,NaN,Four Shot; One Killed; Unidentified shooter in...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male,0::Bernard Gillis,NaN,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://losangeles.cbslocal.com/2013/01/01/man-...,62.0,35.0
2,478855,2013-01-01,Ohio,Lorain,1776 East 28th Street,1,3,http://www.gunviolencearchive.org/incident/478855,http://chronicle.northcoastnow.com/2013/02/14/...,False,9.0,0::Unknown||1::Unknown,0::Unknown||1::Unknown,"Shot - Wounded/Injured||Shot - Dead (murder, a...",41.4455,Cotton Club,-82.1377,2.0,NaN,0::25||1::31||2::33||3::34||4::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||2::Male||3::Male||4::Male,0::Damien Bell||1::Desmen Noble||2::Herman Sea...,NaN,"0::Injured, Unharmed, Arrested||1::Unharmed, A...",0::Subject-Suspect||1::Subject-Suspect||2::Vic...,http://www.morningjournal.com/general-news/201...,56.0,13.0
3,478925,2013-01-05,Colorado,Aurora,16000 block of East Ithaca Place,4,0,http://www.gunviolencearchive.org/incident/478925,http://www.dailydemocrat.com/20130106/aurora-s...,False,6.0,NaN,NaN,"Shot - Dead (murder, accidental, suicide)||Off...",39.6518,NaN,-104.8020,NaN,NaN,0::29||1::33||2::56||3::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Female||1::Male||2::Male||3::Male,0::Stacie Philbrook||1::Christopher Ratliffe||...,NaN,0::Killed||1::Killed||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://denver.cbslocal.com/2013/01/06/officer-...,40.0,28.0
4,478959,2013-01-07,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,http://www.gunviolencearchive.org/incident/478959,http://www.journalnow.com/news/local/article_d...,False,6.0,0::Unknown||1::Unknown,0::Handgun||1::Handgun,"Shot - Wounded/Injured||Shot - Dead (murder, a...",36.1140,NaN,-79.9569,2.0,Two firearms recovered. (Attempted) murder sui...,0::18||1::46||2::14||3::47,0::Adult 18+||1::Adult 18+||2::Teen 12-17||3::...,0::Female||1::Male||2::Male||3::Female,0::Danielle Imani Jameison||1::Maurice Eugene ...,3::Family,0::Injured||1::Injured||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://myfox8.com/2013/01/08/update-mother-sho...,62.0,27.0


In [8]:
def convert_to_dict(value):
    if pd.isna(value):
        return value

    if '||' not in str(value):
        result_dict={0:value}
        return result_dict

    if '||' in str(value):
        pairs = value.split('||')
        pairs[1].strip("'])")

        result_dict = {}
        for pair in pairs:
            #Some are corrupted : 1: instead of ::
            if '::' in pair:
                key, val = pair.split('::', 1)
                result_dict[int(key)] = val
            else:
                key, val = pair.split(':', 1)
                result_dict[int(key)] = val
        return result_dict


updated_base = base.copy()
list_of_dict_columns = ['gun_stolen', 'gun_type', 'participant_age', 'participant_age_group', 'participant_gender', 'participant_name', 'participant_relationship', 'participant_status', 'participant_type']
updated_base[list_of_dict_columns] = updated_base[list_of_dict_columns].applymap(convert_to_dict)
updated_base.head()



,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district
0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,Shot - Wounded/Injured||Mass Shooting (4+ vict...,40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '0::20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",{0: '0::Julian Sims'},NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",http://pittsburgh.cbslocal.com/2013/01/01/4-pe...,NaN,NaN
1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,http://www.dailybulletin.com/article/zz/201301...,False,43.0,NaN,NaN,"Shot - Wounded/Injured||Shot - Dead (murder, a...",33.9090,NaN,-118.3330,NaN,Four Shot; One Killed; Unidentified shooter in...,{0: '0::20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...",{0: '0::Male'},{0: '0::Bernard Gillis'},NaN,"{0: 'Killed', 1: 'Injured', 2: 'Injured', 3: '...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",http://losangeles.cbslocal.com/2013/01/01/man-...,62.0,35.0
2,478855,2013-01-01,Ohio,Lorain,1776 East 28th Street,1,3,http://www.gunviolencearchive.org/incident/478855,http://chronicle.northcoastnow.com/2013/02/14/...,False,9.0,"{0: 'Unknown', 1: 'Unknown'}","{0: 'Unknown', 1: 'Unknown'}","Shot - Wounded/Injured||Shot - Dead (murder, a...",41.4455,Cotton Club,-82.1377,2.0,NaN,"{0: '25', 1: '31', 2: '33', 3: '34', 4: '33'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 2: 'Male', 3: 'Male', 4...","{0: 'Damien Bell', 1: 'Desmen Noble', 2: 'Herm...",NaN,"{0: 'Injured, Unharmed, Arrested', 1: 'Unharme...","{0: 'Subject-Suspect', 1: 'Subject-Suspect', 2...",http://www.morningjournal.com/general-news/201...,56.0,13.0
3,478925,2013-01-05,Colorado,Aurora,16000 block of East Ithaca Place,4,0,http://www.gunviolencearchive.org/incident/478925,http://www.dailydemocrat.com/20130106/aurora-s...,False,6.0,NaN,NaN,"Shot - Dead (murder, accidental, suicide)||Off...",39.6518,NaN,-104.8020,NaN,NaN,"{0: '29', 1: '33', 2: '56', 3: '33'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Female', 1: 'Male', 2: 'Male', 3: 'Male'}","{0: 'Stacie Philbrook', 1: 'Christopher Ratlif...",NaN,"{0: 'Killed', 1: 'Killed', 2: 'Killed', 3: 'Ki...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Su...",http://denver.cbslocal.com/2013/01/06/officer-...,40.0,28.0
4,478959,2013-01-07,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,http://www.gunviolencearchive.org/incident/478959,http://www.journalnow.com/news/local/article_d...,False,6.0,"{0: 'Unknown', 1: 'Unknown'}","{0: 'Handgun', 1: 'Handgun'}","Shot - Wounded/Injured||Shot - Dead (murder, a...",36.1140,NaN,-79.9569,2.0,Two firearms recovered. (Attempted) murder sui...,"{0: '18', 1: '46', 2: '14', 3: '47'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Teen 12-1...","{0: 'Female', 1: 'Male', 2: 'Male', 3: 'Female'}","{0: 'Danielle Imani Jameison', 1: 'Maurice Eug...",{0: '3::Family'},"{0: 'Injured', 1: 'Injured', 2: 'Killed', 3: '...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Su...",http://myfox8.com/2013/01/08/update-mother-sho...,62.0,27.0


In [9]:
def convert_to_dict2(value2):
    if pd.isna(value2):
        return value2

    #some dict are corrupted : one | instead of ||
    list_value=list(value2.values())
    str_value= ''.join(list_value)

    if '|' not in str_value:
        return value2

    if '|' in str_value:
        
        pairs = str_value.split('|')
        pairs[1].strip("'])")

        result_dict = {}
        for pair in pairs:
            #Some are corrupted : 1: instead of ::
            if '::' in pair:
                key, val = pair.split('::', 1)
                result_dict[int(key)] = val
            elif ':' in pair:
                key, val = pair.split(':', 1)
                result_dict[int(key)] = val
            else:
                key, val=0,pair
                result_dict[key]=val
                print(val)
        return result_dict

list_of_dict_columns = ['gun_stolen', 'gun_type', 'participant_age', 'participant_age_group', 'participant_gender', 'participant_name', 'participant_relationship', 'participant_status', 'participant_type']
updated_base[list_of_dict_columns] = updated_base[list_of_dict_columns].applymap(convert_to_dict2)
updated_base.head()


,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district
0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,Shot - Wounded/Injured||Mass Shooting (4+ vict...,40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '0::20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",{0: '0::Julian Sims'},NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",http://pittsburgh.cbslocal.com/2013/01/01/4-pe...,NaN,NaN
1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,http://www.dailybulletin.com/article/zz/201301...,False,43.0,NaN,NaN,"Shot - Wounded/Injured||Shot - Dead (murder, a...",33.9090,NaN,-118.3330,NaN,Four Shot; One Killed; Unidentified shooter in...,{0: '0::20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...",{0: '0::Male'},{0: '0::Bernard Gillis'},NaN,"{0: 'Killed', 1: 'Injured', 2: 'Injured', 3: '...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",http://losangeles.cbslocal.com/2013/01/01/man-...,62.0,35.0
2,478855,2013-01-01,Ohio,Lorain,1776 East 28th Street,1,3,http://www.gunviolencearchive.org/incident/478855,http://chronicle.northcoastnow.com/2013/02/14/...,False,9.0,"{0: 'Unknown', 1: 'Unknown'}","{0: 'Unknown', 1: 'Unknown'}","Shot - Wounded/Injured||Shot - Dead (murder, a...",41.4455,Cotton Club,-82.1377,2.0,NaN,"{0: '25', 1: '31', 2: '33', 3: '34', 4: '33'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 2: 'Male', 3: 'Male', 4...","{0: 'Damien Bell', 1: 'Desmen Noble', 2: 'Herm...",NaN,"{0: 'Injured, Unharmed, Arrested', 1: 'Unharme...","{0: 'Subject-Suspect', 1: 'Subject-Suspect', 2...",http://www.morningjournal.com/general-news/201...,56.0,13.0
3,478925,2013-01-05,Colorado,Aurora,16000 block of East Ithaca Place,4,0,http://www.gunviolencearchive.org/incident/478925,http://www.dailydemocrat.com/20130106/aurora-s...,False,6.0,NaN,NaN,"Shot - Dead (murder, accidental, suicide)||Off...",39.6518,NaN,-104.8020,NaN,NaN,"{0: '29', 1: '33', 2: '56', 3: '33'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Female', 1: 'Male', 2: 'Male', 3: 'Male'}","{0: 'Stacie Philbrook', 1: 'Christopher Ratlif...",NaN,"{0: 'Killed', 1: 'Killed', 2: 'Killed', 3: 'Ki...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Su...",http://denver.cbslocal.com/2013/01/06/officer-...,40.0,28.0
4,478959,2013-01-07,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,http://www.gunviolencearchive.org/incident/478959,http://www.journalnow.com/news/local/article_d...,False,6.0,"{0: 'Unknown', 1: 'Unknown'}","{0: 'Handgun', 1: 'Handgun'}","Shot - Wounded/Injured||Shot - Dead (murder, a...",36.1140,NaN,-79.9569,2.0,Two firearms recovered. (Attempted) murder sui...,"{0: '18', 1: '46', 2: '14', 3: '47'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Teen 12-1...","{0: 'Female', 1: 'Male', 2: 'Male', 3: 'Female'}","{0: 'Danielle Imani Jameison', 1: 'Maurice Eug...",{0: '3::Family'},"{0: 'Injured', 1: 'Injured', 2: 'Killed', 3: '...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Su...",http://myfox8.com/2013/01/08/update-mother-sho...,62.0,27.0


In [10]:
def convert_to_list(value):
    if pd.isna(value):
        return value

    if '||' in str(value):
        liste = value.split('||')
        return liste


list_of_list_columns = ['incident_characteristics', 'sources']
updated_base[list_of_list_columns] = updated_base[list_of_list_columns].applymap(convert_to_list)
updated_base.head()



,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district
0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,"[Shot - Wounded/Injured, Mass Shooting (4+ vic...",40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '0::20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",{0: '0::Julian Sims'},NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://pittsburgh.cbslocal.com/2013/01/01/4-p...,NaN,NaN
1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,http://www.dailybulletin.com/article/zz/201301...,False,43.0,NaN,NaN,"[Shot - Wounded/Injured, Shot - Dead (murder, ...",33.9090,NaN,-118.3330,NaN,Four Shot; One Killed; Unidentified shooter in...,{0: '0::20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...",{0: '0::Male'},{0: '0::Bernard Gillis'},NaN,"{0: 'Killed', 1: 'Injured', 2: 'Injured', 3: '...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://losangeles.cbslocal.com/2013/01/01/man...,62.0,35.0
2,478855,2013-01-01,Ohio,Lorain,1776 East 28th Street,1,3,http://www.gunviolencearchive.org/incident/478855,http://chronicle.northcoastnow.com/2013/02/14/...,False,9.0,"{0: 'Unknown', 1: 'Unknown'}","{0: 'Unknown', 1: 'Unknown'}","[Shot - Wounded/Injured, Shot - Dead (murder, ...",41.4455,Cotton Club,-82.1377,2.0,NaN,"{0: '25', 1: '31', 2: '33', 3: '34', 4: '33'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 2: 'Male', 3: 'Male', 4...","{0: 'Damien Bell', 1: 'Desmen Noble', 2: 'Herm...",NaN,"{0: 'Injured, Unharmed, Arrested', 1: 'Unharme...","{0: 'Subject-Suspect', 1: 'Subject-Suspect', 2...",[http://www.morningjournal.com/general-news/20...,56.0,13.0
3,478925,2013-01-05,Colorado,Aurora,16000 block of East Ithaca Place,4,0,http://www.gunviolencearchive.org/incident/478925,http://www.dailydemocrat.com/20130106/aurora-s...,False,6.0,NaN,NaN,"[Shot - Dead (murder, accidental, suicide), Of...",39.6518,NaN,-104.8020,NaN,NaN,"{0: '29', 1: '33', 2: '56', 3: '33'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Female', 1: 'Male', 2: 'Male', 3: 'Male'}","{0: 'Stacie Philbrook', 1: 'Christopher Ratlif...",NaN,"{0: 'Killed', 1: 'Killed', 2: 'Killed', 3: 'Ki...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Su...",[http://denver.cbslocal.com/2013/01/06/officer...,40.0,28.0
4,478959,2013-01-07,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,http://www.gunviolencearchive.org/incident/478959,http://www.journalnow.com/news/local/article_d...,False,6.0,"{0: 'Unknown', 1: 'Unknown'}","{0: 'Handgun', 1: 'Handgun'}","[Shot - Wounded/Injured, Shot - Dead (murder, ...",36.1140,NaN,-79.9569,2.0,Two firearms recovered. (Attempted) murder sui...,"{0: '18', 1: '46', 2: '14', 3: '47'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Teen 12-1...","{0: 'Female', 1: 'Male', 2: 'Male', 3: 'Female'}","{0: 'Danielle Imani Jameison', 1: 'Maurice Eug...",{0: '3::Family'},"{0: 'Injured', 1: 'Injured', 2: 'Killed', 3: '...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Su...",[http://myfox8.com/2013/01/08/update-mother-sh...,62.0,27.0


In [11]:
updated_base['participant_type'][0]

{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Victim', 4: 'Subject-Suspect'}

### Début des statistiques descriptives 

import des packages nécessaires:

In [12]:
from plotly.offline import init_notebook_mode
init_notebook_mode(connected= True)

In [13]:
import plotly.express as px

### Barplot décrivant le nombre d'incidents par arme à feu par Etat

In [14]:
fig = px.bar(updated_base, x=pd.unique(updated_base['state']), y=updated_base.groupby('state').count()['incident_id'], title="Nombre d'incidents par arme à feu par Etat",
labels={"x":"Etat", "y":"nombre d'incidents"})

In [15]:
fig.show()

### Graphique étudiant le nombre d'incidents par mois 

In [16]:
updated_base['date']=pd.to_datetime(updated_base['date'])

In [17]:
updated_base['month_year'] =updated_base['date'].dt.month.astype(str)+"-"+updated_base['date'].dt.year.astype(str)

In [18]:

fig2 = px.bar(updated_base, x=pd.unique(updated_base['month_year']), y=updated_base.groupby(updated_base['month_year']).count()['incident_id'],
              labels={"x":"date", "y":"nombre d'incidents"},
              title="Nombre d'incidents par arme à feu par mois")

In [19]:
fig2

On remarque que tous les 5 ou 6 mois très peu d'incidents sont renseignés : problème de récolte des données ? saisonnalité peu probable au vu de la variation des mois concernés

### étude du nombre de morts et blessés par incident

In [20]:
import plotly.graph_objects as go


In [21]:
fig3 = px.bar(updated_base, x=pd.unique(updated_base['n_killed']), y=updated_base.groupby(updated_base['n_killed']).count()['incident_id'],
              labels={"x":"nombre de morts", "y":"nombre d'incidents (échelle logarithmique)"},
              title="Nombre d'incidents ayant entraîné des morts",
              log_y=True)

fig3

In [22]:
fig4 = px.bar(updated_base, x=pd.unique(updated_base['n_injured']), y=updated_base.groupby(updated_base['n_injured']).count()['incident_id'],
              labels={"x":"nombre de blessés", "y":"nombre d'incidents (échelle logarithmique)"},
              title="Nombre de blessés par incident",
              log_y=True)

fig4

### statistiques descriptives sur les suspects et auteurs des incidents 

In [23]:
liste=[]

for i in updated_base['participant_status']:
    if type(i)!=dict:
        continue
    else:
        liste.append(i.values())

flat_list = [item for sublist in liste for item in sublist]
flat_list
pd.unique(flat_list)

array(['Arrested', 'Injured', 'Killed', 'Injured, Unharmed, Arrested',
       'Unharmed, Arrested', 'Unharmed', 'Injured, Arrested',
       'Killed, Unharmed, Arrested', 'Injured, Unharmed',
       'Killed, Injured', '0::Injured', '0::Killed', '0::Unharmed',
       '0::Unharmed, Arrested', '1::Unharmed', '1::Killed',
       'Killed, Unharmed', '0::Injured, Arrested', 'Killed, Arrested',
       '1::Injured', '1::Unharmed, Arrested', '0:Injured', '0:Killed',
       '0:Unharmed', '1:Unharmed', '0:Unharmed, Arrested',
       '1:Unharmed, Arrested', '1:Killed', '0:Arrested', '1:Injured',
       '2::Unharmed', '0::Arrested', '2::Killed', '4::Unharmed, Arrested',
       '2::Arrested', '3::Unharmed', '4::Unharmed', '0::Killed, Unharmed',
       '1::Injured, Arrested', '2::Unharmed, Arrested',
       '0::Killed, Arrested', '2::Injured', '1::Arrested',
       '0::Injured, Unharmed', '0::Injured, Unharmed, Arrested',
       '0::Killed, Unharmed, Arrested', '3::Unharmed, Arrested',
       '2::Inju

In [24]:
liste2=[]

for i in updated_base['participant_type']:
    if type(i)!=dict:
        continue
    else:
        liste2.append(i.values())

flat_list2 = [item for sublist in liste2 for item in sublist]
print(pd.unique(flat_list2))

['Victim' 'Subject-Suspect' '0::Subject-Suspect' '0::Victim' '0:Victim'
 '0:Subject-Suspect']


In [25]:
liste3=[]

for i in updated_base['participant_age_group']:
    if type(i)!=dict:
        continue
    else:
        liste3.append(i.values())

flat_list3= [item for sublist in liste3 for item in sublist]
pd.unique(flat_list3)

array(['Adult 18+', 'Teen 12-17', 'Child 0-11', '0::Adult 18+',
       '1::Adult 18+', '4::Adult 18+', '5::Adult 18+', '3::Child 0-11',
       '8::Teen 12-17', '0::Teen 12-17', '2::Adult 18+', '0::Child 0-11',
       '2::Teen 12-17', '1::Teen 12-17', '3::Adult 18+', '1::Child 0-11',
       '8::Adult 18+', '3::Teen 12-17', '0:Adult 18+', '0:Teen 12-17',
       '0:Child 0-11', '1:Teen 12-17', '1:Adult 18+', '3:Adult 18+',
       '2:Adult 18+', '2:Teen 12-17', '13::Adult 18+', '7::Adult 18+',
       '2::Child 0-11', '6::Adult 18+', '6::Teen 12-17', '9::Adult 18+',
       '4::Teen 12-17'], dtype=object)

In [39]:
liste7=[]

for i in updated_base['participant_age']:
    if type(i)!=dict:
        continue
    else:
        liste7.append(i.values())

flat_list7 = [item for sublist in liste7 for item in sublist]
pd.unique(flat_list7)

array(['0::20', '25', '31', '33', '34', '29', '56', '18', '46', '14',
       '47', '23', '55', '51', '40', '9', '5', '2', '15', '0::15',
       '0::19', '0::28', '78', '48', '28', '5::24', '20', '19', '41',
       '22', '21', '0::22', '39', '68', '35', '26', '69', '27', '37',
       '8::29', '17', '50', '42', '24', '16', '13', '57', '66', '67',
       '62', '64', '36', '43', '0::23', '30', '32', '1', '3', '63', '10',
       '4::29', '53', '4::47', '1::22', '60', '58', '4::22', '72', '45',
       '49', '12', '44', '70', '0::18', '0::25', '0::27', '0', '54', '6',
       '38', '5::18', '8', '4', '7', '52', '79', '4::28', '1::30', '59',
       '11', '0::31', '1::25', '4::20', '5::20', '80', '0::32', '4::35',
       '73', '61', '3::11', '4::26', '0::21', '8::15', '4::32', '0::47',
       '4::36', '0::17', '1::24', '0::24', '85', '0::13', '0::34',
       '0::52', '0::60', '0::46', '0::35', '0::8', '88', '1::21', '0::39',
       '0::33', '2::19', '1::23', '0::61', '0::49', '3::25', '2::24',
 

In [26]:
liste4=[]

for i in updated_base['participant_gender']:
    if type(i)!=dict:
        continue
    else:
        liste4.append(i.values())

flat_list4 = [item for sublist in liste4 for item in sublist]
pd.unique(flat_list4)

array(['Male', 'Female', '0::Male', '4::Female', '4::Male', '1::Male',
       '3::Male', '0::Female', '2::Male', '1::Female', '8::Male',
       '5::Male', '3::Female', '6::Male', '0:Male', '1:Male', '0:Female',
       '2:Male', '3:Male', '1:Female', '5:Female', '3:Female', '4:Male',
       '2:Female', '2::Female', '13::Male', '0::Male, female', '7::Male',
       '5::Female', '10::Male'], dtype=object)

In [28]:
liste5=[]

for i in updated_base['gun_type']:
    if type(i)!=dict:
        continue
    else:
        liste5.append(i.values())

flat_list5 = [item for sublist in liste5 for item in sublist]
pd.unique(flat_list5)

array(['Unknown', 'Handgun', '22 LR', '223 Rem [AR-15]', '0::Unknown',
       '0::Shotgun', '0::Handgun', '0::9mm', '0::45 Auto', '0::12 gauge',
       '0::7.62 [AK-47]', '9mm', '40 SW', 'Shotgun', '7.62 [AK-47]',
       '45 Auto', '0::44 Mag', '44 Mag', '0::Other', '0::40 SW',
       '0::38 Spl', '0::22 LR', '0::380 Auto', '410 gauge', '32 Auto',
       '0::308 Win', 'Other', '0::Rifle', '0::357 Mag', 'Rifle',
       '0::16 gauge', '357 Mag', '38 Spl', '0::30-30 Win', '25 Auto',
       '12 gauge', '30-30 Win', '0::410 gauge', '0::223 Rem [AR-15]',
       '0::32 Auto', '0:Unknown', '0:22 LR', '0:25 Auto', '0:9mm',
       '0:Handgun', '380 Auto', '0:45 Auto', '0:Shotgun', '0:20 gauge',
       '0:357 Mag', '0:12 gauge', '0:38 Spl', '0:40 SW', '0:380 Auto',
       '0:44 Mag', '308 Win', '0:32 Auto', '0:Other', '20 gauge',
       '16 gauge', '0::10mm', '0::25 Auto', '0::20 gauge', '30-06 Spr',
       '0::30-06 Spr', '0::300 Win', '300 Win', '10mm', '0::28 gauge',
       '28 gauge'], dtype=

In [29]:
liste6=[]

for i in updated_base['gun_stolen']:
    if type(i)!=dict:
        continue
    else:
        liste6.append(i.values())

flat_list6 = [item for sublist in liste6 for item in sublist]
pd.unique(flat_list6)

array(['Unknown', '0::Unknown', 'Not-stolen', 'Stolen', '0::Not-stolen',
       '0::Stolen', '0:Unknown', '0:Not-stolen', '0:Stolen'], dtype=object)

Les dictionnaires ne sont pas parfaits au moment où je commence ces statistiques au vu des valeurs uniques mais environ 15 erreurs sur 239 000 observations me paraît tout à fait raisonnable.

In [37]:
data_suspects=[]
index=0

for i in updated_base['participant_type']:
    if type(i)==dict and len(i)>0:
        for key,value in i.items():
            if value=='Subject-Suspect':
                result_dict={'id':index}

                if type(updated_base['participant_age_group'][index])==dict:
                    if key in updated_base['participant_age_group'][index].keys():
                        result_dict['age_group_sus']=updated_base['participant_age_group'][index][key]
                else:
                    result_dict['age_group_sus']=np.nan
                
                if type(updated_base['participant_age'][index])==dict:
                    if key in updated_base['participant_age'][index].keys():
                        result_dict['age_sus']=updated_base['participant_age'][index][key]
                else:
                    result_dict['age_sus']=np.nan

                if type(updated_base['participant_gender'][index])==dict:
                    if key in updated_base['participant_gender'][index].keys():
                        result_dict['genre_sus']=updated_base['participant_gender'][index][key]
                else:
                    result_dict['genre_sus']=np.nan

                if type(updated_base['participant_status'][index])==dict:
                    if key in updated_base['participant_status'][index].keys():
                        result_dict['status_sus']=updated_base['participant_status'][index][key]
                else:
                    result_dict['status_sus']=np.nan

                if type(updated_base['gun_type'][index])==dict:
                    if key in updated_base['gun_type'][index].keys():
                        result_dict['gun_type']=updated_base['gun_type'][index][key]
                else:
                    result_dict['gun_type']=np.nan

                if type(updated_base['gun_stolen'][index])==dict:
                    if key in updated_base['gun_stolen'][index].keys():
                        result_dict['gun_stolen']=updated_base['gun_stolen'][index][key]
                else:
                    result_dict['gun_stolen']=np.nan


                data_suspects.append(result_dict)

    elif np.isnan(i)==False:
        if updated_base['participant_type'][i]=='Subject-Suspect':
            result_dict={'id':index}

            if type(updated_base['participant_age_group'][index])==dict:
                if key in updated_base['participant_age_group'][index].keys():
                        result_dict['age_group_sus']=updated_base['participant_age_group'][index][key]
            else:
                result_dict['age_group_sus']=np.nan
            
            if type(updated_base['participant_age'][index])==dict:
                if key in updated_base['participant_age'][index].keys():
                    result_dict['age_sus']=updated_base['participant_age'][index][key]
            else:
                result_dict['age_sus']=np.nan

            if type(updated_base['participant_gender'][index])==dict:
                if key in updated_base['participant_gender'][index].keys():
                    result_dict['genre_sus']=updated_base['participant_gender'][index][key]
            else:
                result_dict['genre_sus']=np.nan

            if type(updated_base['participant_status'][index])==dict:
                if key in updated_base['participant_status'][index].keys():
                    result_dict['status_sus']=updated_base['participant_status'][index][key]
            else:
                result_dict['status_sus']=np.nan

            if type(updated_base['gun_type'][index])==dict:
                if key in updated_base['gun_type'][index].keys():
                    result_dict['gun_type']=updated_base['gun_type'][index][key]
            else:
                result_dict['gun_type']=np.nan

            if type(updated_base['gun_stolen'][index])==dict:
                if key in updated_base['gun_stolen'][index].keys():
                    result_dict['gun_stolen']=updated_base['gun_stolen'][index][key]
            else:
                result_dict['gun_stolen']=np.nan
            data_suspects.append(result_dict)

    index+=1


df_suspects=pd.DataFrame(data_suspects)

In [38]:
df_suspects.head()

,id,age_group_sus,genre_sus,status_sus,gun_type,gun_stolen,age_sus
0,0,Adult 18+,Female,Injured,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Adult 18+,Male,"Injured, Unharmed, Arrested",Unknown,Unknown,25
3,2,Adult 18+,Male,"Unharmed, Arrested",Unknown,Unknown,31
4,3,Adult 18+,Male,Killed,NaN,NaN,33


On obtient ici un dataframe permettant d'obtenir l'âge, le genre et le status de tous les participants aux incidents caractérisés comme 'Subject-Suspect' afin de pouvoir faire des statistiques descriptives. La colonne 'id' corresponds à l'index de l'incident concerné dans la base updated_base.

In [34]:
#il reste environ 10 erreurs de label concernant l'âge, on se permet donc d'ignorer le problème en faisant sur une base filtrée n'ayant que les bonnes catégories
groupes=['Adult 18+', 'Teen 12-17','Child 0-11']
filtered_suspects_age=df_suspects[df_suspects['age_group_sus'].isin(groupes)]

fig5 = px.bar(filtered_suspects_age, x=sorted(pd.unique(filtered_suspects_age['age_group_sus'])), y=filtered_suspects_age.groupby('age_group_sus').count()['id'],
              labels={"x":"âge du suspect", "y":"nombre de suspects"},
              title="Nombre de suspect par tranche d'âge")

fig5

In [73]:
age=list(range(10,90))
age=[str(num) for num in age]

filtered_suspects_age2=df_suspects[df_suspects['age_sus'].isin(age)]
filtered_suspects_age2

fig8 = px.bar(filtered_suspects_age2, x=sorted(pd.unique(filtered_suspects_age2['age_sus'])), y=filtered_suspects_age2.groupby('age_sus').count()['id'],
            labels={"x":"âge du suspect", "y":"nombre de suspects"},
            title="Nombre de suspect par tranche d'âge")

fig8

In [56]:
#il reste environ 10 erreurs de label concernant le genre, on se permet donc d'ignorer le problème en faisant sur une base filtrée n'ayant que les bonnes catégories
genre=['Male', 'Female']
filtered_suspects_genre=df_suspects[df_suspects['genre_sus'].isin(genre)]

fig6 = px.bar(filtered_suspects_genre, x=sorted(pd.unique(filtered_suspects_genre['genre_sus'])), y=filtered_suspects_genre.groupby('genre_sus').count()['id'],
              labels={"x":"genre du suspect", "y":"nombre de suspects"},
              title="Nombre de suspect par genre")

fig6

In [59]:
#il reste environ 10 erreurs de label concernant le statut, on se permet donc d'ignorer le problème en faisant sur une base filtrée n'ayant que les bonnes catégories
statut=['Injured','Injured, Unharmed, Arrested',
       'Unharmed, Arrested', 'Killed', 'Unharmed', 'Injured, Arrested',
       'Killed, Unharmed, Arrested', 'Arrested', 'Injured, Unharmed','Killed, Unharmed',
       'Killed, Arrested', 'Killed, Injured']
filtered_suspects_status=df_suspects[df_suspects['status_sus'].isin(statut)]

fig7 = px.bar(filtered_suspects_status, x=sorted(pd.unique(filtered_suspects_status['status_sus'])), y=filtered_suspects_status.groupby('status_sus').count()['id'],
              labels={"x":"statut du suspect", "y":"nombre de suspects (échelle logarithmique)"},
              title="Nombre de suspect par statut",
              log_y=True)

fig7

In [58]:
fig9 = px.bar(filtered_suspects_status, x=sorted(pd.unique(filtered_suspects_status['status_sus'])), y=filtered_suspects_status.groupby('status_sus').count()['id'],
              labels={"x":"statut du suspect", "y":"nombre de suspects"},
              title="Nombre de suspect par statut")

fig9

### Statistiques sur les caractéristiques des incidents 

In [60]:
liste_carac=[]
for i in updated_base["incident_characteristics"]:
    if type(i) != list:
        pass
    else:
        liste_carac.append(i)
flat_carac=[item for sublist in liste_carac for item in sublist]
pd.unique(flat_carac)

array(['Shot - Wounded/Injured',
       'Mass Shooting (4+ victims injured or killed excluding the subject/suspect/perpetrator, one location)',
       'Possession (gun(s) found during commission of other crimes)',
       'Possession of gun by felon or prohibited person',
       'Shot - Dead (murder, accidental, suicide)', 'Gang involvement',
       'Shots Fired - No Injuries',
       'Bar/club incident - in or around establishment',
       'Officer Involved Incident',
       'Officer Involved Shooting - subject/suspect/perpetrator killed',
       'Drug involvement', 'Kidnapping/abductions/hostage',
       'Under the influence of alcohol or drugs (only applies to the subject/suspect/perpetrator )',
       'Suicide^', 'Murder/Suicide',
       'Attempted Murder/Suicide (one variable unsuccessful)',
       'Domestic Violence', 'Home Invasion',
       'Home Invasion - Resident killed',
       'Armed robbery with injury/death and/or evidence of DGU found',
       'Drive-by (car to street, ca

On voit ici que ces données liées aux caractéristiques sont des champs libres non-normalisés donc on va d'abord s'intéresser aux descriptions comportant la mention de suicides car c'est, selon le Centers for Disease Control and Prevention's (CDC) National Center for Health Statistics, la cause principale de mort lors des incidents par arme à feu.

In [61]:
#création d'une indicatrice signalant la description d'un suicide

liste_indic=[]

for i in updated_base['incident_characteristics']:
    if i is not None:
        if "suicide" in str(i) :
            liste_indic.append(1)
        else:
            liste_indic.append(0)
    else:
        liste_indic.append(0)
    

updated_base['indic_suicide']=liste_indic
updated_base.head()

,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district,month_year,indic_suicide
0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,"[Shot - Wounded/Injured, Mass Shooting (4+ vic...",40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '0::20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",{0: '0::Julian Sims'},NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://pittsburgh.cbslocal.com/2013/01/01/4-p...,NaN,NaN,1-2013,0
1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,http://www.dailybulletin.com/article/zz/201301...,False,43.0,NaN,NaN,"[Shot - Wounded/Injured, Shot - Dead (murder, ...",33.9090,NaN,-118.3330,NaN,Four Shot; One Killed; Unidentified shooter in...,{0: '0::20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...",{0: '0::Male'},{0: '0::Bernard Gillis'},NaN,"{0: 'Killed', 1: 'Injured', 2: 'Injured', 3: '...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://losangeles.cbslocal.com/2013/01/01/man...,62.0,35.0,1-2013,1
2,478855,2013-01-01,Ohio,Lorain,1776 East 28th Street,1,3,http://www.gunviolencearchive.org/incident/478855,http://chronicle.northcoastnow.com/2013/02/14/...,False,9.0,"{0: 'Unknown', 1: 'Unknown'}","{0: 'Unknown', 1: 'Unknown'}","[Shot - Wounded/Injured, Shot - Dead (murder, ...",41.4455,Cotton Club,-82.1377,2.0,NaN,"{0: '25', 1: '31', 2: '33', 3: '34', 4: '33'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 2: 'Male', 3: 'Male', 4...","{0: 'Damien Bell', 1: 'Desmen Noble', 2: 'Herm...",NaN,"{0: 'Injured, Unharmed, Arrested', 1: 'Unharme...","{0: 'Subject-Suspect', 1: 'Subject-Suspect', 2...",[http://www.morningjournal.com/general-news/20...,56.0,13.0,1-2013,1
3,478925,2013-01-05,Colorado,Aurora,16000 block of East Ithaca Place,4,0,http://www.gunviolencearchive.org/incident/478925,http://www.dailydemocrat.com/20130106/aurora-s...,False,6.0,NaN,NaN,"[Shot - Dead (murder, accidental, suicide), Of...",39.6518,NaN,-104.8020,NaN,NaN,"{0: '29', 1: '33', 2: '56', 3: '33'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Female', 1: 'Male', 2: 'Male', 3: 'Male'}","{0: 'Stacie Philbrook', 1: 'Christopher Ratlif...",NaN,"{0: 'Killed', 1: 'Killed', 2: 'Killed', 3: 'Ki...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Su...",[http://denver.cbslocal.com/2013/01/06/officer...,40.0,28.0,1-2013,1
4,478959,2013-01-07,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,http://www.gunviolencearchive.org/incident/478959,http://www.journalnow.com/news/local/article_d...,False,6.0,"{0: 'Unknown', 1: 'Unknown'}","{0: 'Handgun', 1: 'Handgun'}","[Shot - Wounded/Injured, Shot - Dead (murder, ...",36.1140,NaN,-79.9569,2.0,Two firearms recovered. (Attempted) murder sui...,"{0: '18', 1: '46', 2: '14', 3: '47'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Teen 12-1...","{0: 'Female', 1: 'Male', 2: 'Male', 3: 'Female'}","{0: 'Danielle Imani Jameison', 1: 'Maurice Eug...",{0: '3::Family'},"{0: 'Injured', 1: 'Injured', 2: 'Killed', 3: '...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Su...",[http://myfox8.com/2013/01/08/update-mother-sh...,62.0,27.0,1-2013,1


In [62]:
fig10 = px.bar(updated_base[updated_base['n_killed']>=1], x=pd.unique(updated_base['indic_suicide']), y=updated_base[updated_base['n_killed']>=1].groupby(updated_base['indic_suicide']).count()['incident_id'],
              labels={"x":"incident impliquant un suicide (1) ou non (0)", "y":"nombre d'incidents"},
              title="Parmi les incidents impliquant au moins un mort, combien sont le résultat d'un suicide ?")

fig10

### Statistiques descriptives sur les victimes des incidents 

On commence par recréer une base similaire à celle des suspects pour récupérer les informations présentes dans les dictionnaires.

In [66]:
data_victimes=[]
index=0

for i in updated_base['participant_type']:
    if type(i)==dict and len(i)>0:
        for key,value in i.items():
            if value=='Victim':
                result_dict={'id':index}

                if type(updated_base['participant_age_group'][index])==dict:
                    if key in updated_base['participant_age_group'][index].keys():
                        result_dict['age_group_vict']=updated_base['participant_age_group'][index][key]
                else:
                    result_dict['age_group_vict']=np.nan
                
                if type(updated_base['participant_age'][index])==dict:
                    if key in updated_base['participant_age'][index].keys():
                        result_dict['age_vict']=updated_base['participant_age'][index][key]
                else:
                    result_dict['age_vict']=np.nan

                if type(updated_base['participant_gender'][index])==dict:
                    if key in updated_base['participant_gender'][index].keys():
                        result_dict['genre_vict']=updated_base['participant_gender'][index][key]
                else:
                    result_dict['genre_vict']=np.nan

                if type(updated_base['participant_status'][index])==dict:
                    if key in updated_base['participant_status'][index].keys():
                        result_dict['status_vict']=updated_base['participant_status'][index][key]
                else:
                    result_dict['status_vict']=np.nan

                if type(updated_base['gun_type'][index])==dict:
                    if key in updated_base['gun_type'][index].keys():
                        result_dict['gun_type']=updated_base['gun_type'][index][key]
                else:
                    result_dict['gun_type']=np.nan

                if type(updated_base['gun_stolen'][index])==dict:
                    if key in updated_base['gun_stolen'][index].keys():
                        result_dict['gun_stolen']=updated_base['gun_stolen'][index][key]
                else:
                    result_dict['gun_stolen']=np.nan
                
                data_victimes.append(result_dict)


    elif np.isnan(i)==False:
        if updated_base['participant_type'][i]=='Victim':
            result_dict={'id':index}
            if type(updated_base['participant_age_group'][index])==dict:
                    if key in updated_base['participant_age_group'][index].keys():
                        result_dict['age_group_vict']=updated_base['participant_age_group'][index][key]
            else:
                result_dict['age_group_vict']=np.nan
            
            if type(updated_base['participant_age'][index])==dict:
                if key in updated_base['participant_age'][index].keys():
                    result_dict['age_vict']=updated_base['participant_age'][index][key]
            else:
                result_dict['age_vict']=np.nan

            if type(updated_base['participant_gender'][index])==dict:
                if key in updated_base['participant_gender'][index].keys():
                    result_dict['genre_vict']=updated_base['participant_gender'][index][key]
            else:
                result_dict['genre_vict']=np.nan

            if type(updated_base['participant_status'][index])==dict:
                if key in updated_base['participant_status'][index].keys():
                    result_dict['status_vict']=updated_base['participant_status'][index][key]
            else:
                result_dict['status_vict']=np.nan

            if type(updated_base['gun_type'][index])==dict:
                if key in updated_base['gun_type'][index].keys():
                    result_dict['gun_type']=updated_base['gun_type'][index][key]
            else:
                result_dict['gun_type']=np.nan

            if type(updated_base['gun_stolen'][index])==dict:
                if key in updated_base['gun_stolen'][index].keys():
                    result_dict['gun_stolen']=updated_base['gun_stolen'][index][key]
            else:
                result_dict['gun_stolen']=np.nan


            data_victimes.append(result_dict)

    index+=1


df_victim=pd.DataFrame(data_victimes)

In [68]:
df_victim.head()

,id,age_group_vict,age_vict,genre_vict,status_vict,gun_type,gun_stolen
0,0,Adult 18+,0::20,Male,Arrested,NaN,NaN
1,0,Adult 18+,NaN,Male,Injured,NaN,NaN
2,0,Adult 18+,NaN,NaN,Injured,NaN,NaN
3,0,Adult 18+,NaN,Male,Injured,NaN,NaN
4,1,Adult 18+,0::20,0::Male,Killed,NaN,NaN


On va faire des graphiques de statistiques descriptives similaires à ceux concernant les suspects:

In [69]:
#il reste environ 10 erreurs de label concernant l'âge, on se permet donc d'ignorer le problème en faisant sur une base filtrée n'ayant que les bonnes catégories
groupes=['Adult 18+', 'Teen 12-17','Child 0-11']
filtered_victim_age=df_victim[df_victim['age_group_vict'].isin(groupes)]

fig11 = px.bar(filtered_victim_age, x=sorted(pd.unique(filtered_victim_age['age_group_vict'])), y=filtered_victim_age.groupby('age_group_vict').count()['id'],
              labels={"x":"âge de la victime", "y":"nombre de victimes"},
              title="Nombre de victimes par tranche d'âge")

fig11

In [70]:
#il reste environ 10 erreurs de label concernant le genre, on se permet donc d'ignorer le problème en faisant sur une base filtrée n'ayant que les bonnes catégories
genre=['Male', 'Female']
filtered_victim_genre=df_victim[df_victim['genre_vict'].isin(genre)]

fig12 = px.bar(filtered_victim_genre, x=sorted(pd.unique(filtered_victim_genre['genre_vict'])), y=filtered_victim_genre.groupby('genre_vict').count()['id'],
              labels={"x":"genre de la victime", "y":"nombre de victimes"},
              title="Genre des victimes")

fig12

In [71]:
#il reste environ 10 erreurs de label concernant le statut, on se permet donc d'ignorer le problème en faisant sur une base filtrée n'ayant que les bonnes catégories
stat_vict=['Arrested', 'Injured', 'Killed', 'Unharmed',
       'Injured, Unharmed', 'Killed, Injured', 'Injured, Arrested', 'Unharmed, Arrested', 'Killed, Arrested',
       'Killed, Unharmed','Killed, Unharmed, Arrested', 'Injured, Unharmed, Arrested']
filtered_vict_status=df_victim[df_victim['status_vict'].isin(statut)]

fig13 = px.bar(filtered_vict_status, x=sorted(pd.unique(filtered_vict_status['status_vict'])), y=filtered_vict_status.groupby('status_vict').count()['id'],
              labels={"x":"statut de la victime", "y":"nombre de victimes"},
              title="Statut des victimes")

fig13

In [72]:
fig14 = px.bar(filtered_vict_status, x=sorted(pd.unique(filtered_vict_status['status_vict'])), y=filtered_vict_status.groupby('status_vict').count()['id'],
              labels={"x":"statut de la victime", "y":"nombre de victimes (échelle logarithmique)"},
              title="Statut des victimes", log_y=True)

fig14

In [75]:
age=list(range(100))
age=[str(num) for num in age]

filtered_vict_age2=df_victim[df_victim['age_vict'].isin(age)]
filtered_vict_age2

fig15 = px.bar(filtered_vict_age2, x=sorted(pd.unique(filtered_vict_age2['age_vict'])), y=filtered_vict_age2.groupby('age_vict').count()['id'],
            labels={"x":"âge de la victime", "y":"nombre de victimes"},
            title="Nombre de victimes par âge")

fig15

### Tentative de mise en avant de corrélations sans base supplémentaire : cf stat_des_elo_corr